In [202]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
from yahooquery import Ticker

In [203]:
year = 2018
df = pd.read_csv(f'Data/sp500_wrds_{year}_fundamental.csv')
price_df = pd.read_csv(f'Data/sp500_wrds_{year}_price.csv') # include 12 months before data
# df.head()

In [204]:
price_df.columns

Index(['tic', 'datadate', 'gvkey', 'ggroup', 'gind', 'gsector', 'cshtrm',
       'prccm'],
      dtype='object')

In [205]:
len(df.columns)

76

In [206]:
# initial_columns = ['permno', 'adate', 'qdate', 'public_date', 'bm', 'evm', 'pe_op_basic',
#        'pe_op_dil', 'pe_exi', 'ps', 'pcf', 'dpr', 'npm', 'opmad', 'gpm',
#        'ptpm', 'cfm', 'roa', 'roe', 'roce', 'efftax', 'aftret_eq',
#        'aftret_invcapx', 'aftret_equity', 'pretret_noa', 'GProf',
#        'equity_invcap', 'debt_invcap', 'totdebt_invcap', 'capital_ratio',
#        'int_debt', 'int_totdebt', 'cash_lt', 'invt_act', 'rect_act', 'debt_at',
#        'debt_ebitda', 'short_debt', 'curr_debt', 'lt_debt', 'profit_lct',
#        'ocf_lct', 'cash_debt', 'fcf_ocf', 'lt_ppent', 'dltt_be', 'debt_assets',
#        'debt_capital', 'de_ratio', 'intcov', 'intcov_ratio', 'inv_turn',
#        'at_turn', 'rect_turn', 'pay_turn', 'sale_equity', 'sale_nwc',
#        'rd_sale', 'adv_sale', 'staff_sale', 'accrual', 'ptb', 'PEG_trailing',
#        'divyield', 'TICKER']
# # some are dropped due to high missing values or redundancy after this block

# df = df[initial_columns]

In [207]:
cols_to_drop = ['permno','adate', 'qdate','PEG_trailing','inv_turn', 'sale_nwc','profit_lct','ocf_lct','curr_debt', 'pretret_noa', 'intcov', 'intcov_ratio', 'rect_act','invt_act']

In [208]:
df = df.drop(columns=cols_to_drop)
df = df.rename(columns={'TICKER': 'ticker', 'public_date': 'date'})
df = df.dropna(subset=['ticker'])

In [209]:
# df.isna().sum()
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
# df.set_index('date', inplace=True)

In [210]:
#remoce % sign from divyield and convert to float
df["divyield"] = df["divyield"].str.rstrip('%')
df["divyield"] = df["divyield"].fillna(0)
df["divyield"] = df["divyield"].astype('float')

In [211]:
df = df.sort_values(['ticker', 'date'])
df = df.groupby('ticker').apply(lambda x: x.ffill().bfill())
df = df.reset_index(drop=True)

In [212]:
df.head()

,gvkey,date,CAPEI,bm,evm,pe_op_basic,pe_op_dil,pe_exi,pe_inc,ps,...,sale_invcap,sale_equity,rd_sale,adv_sale,staff_sale,accrual,ptb,divyield,ticker,cusip
0,126554,2018-01-31,42.175,0.224,21.840,33.226,33.683,34.967,34.967,5.304,...,0.674,0.926,0.076,0.008,0.0,-0.025,4.833,0.812,A,00846U10
1,126554,2018-02-28,39.358,0.224,21.840,31.036,31.463,32.662,32.662,4.950,...,0.674,0.926,0.076,0.008,0.0,-0.025,4.510,0.869,A,00846U10
2,126554,2018-03-31,42.198,0.191,20.567,28.590,29.214,113.390,113.390,4.674,...,0.723,1.007,0.077,0.008,0.0,-0.096,4.771,0.891,A,00846U10
3,126554,2018-04-30,41.466,0.191,20.567,28.094,28.707,111.424,111.424,4.593,...,0.723,1.007,0.077,0.008,0.0,-0.096,4.688,0.907,A,00846U10
4,126554,2018-05-31,38.751,0.191,20.567,26.462,27.039,104.949,104.949,4.292,...,0.723,1.007,0.077,0.008,0.0,-0.096,4.381,0.963,A,00846U10


In [213]:
df.shape

(5347, 62)

2022 specific

In [214]:
# renam fb ticker to meta (2022 change)
df['ticker'] = df['ticker'].replace({'FB': 'META'})

In [215]:
print(len(df["ticker"].unique()))

466


In [216]:
price_df.columns

Index(['tic', 'datadate', 'gvkey', 'ggroup', 'gind', 'gsector', 'cshtrm',
       'prccm'],
      dtype='object')

In [217]:
# create a dictionary to map ticker to gind
gind_mapping = price_df.set_index('tic')['gind'].to_dict()
gsec_mapping = price_df.set_index('tic')['gsector'].to_dict()
df['gind'] = df['ticker'].map(gind_mapping)
df['gsector'] = df['ticker'].map(gsec_mapping)


price_df = price_df[['tic', 'datadate', 'cshtrm', 'prccm']]
price_df = price_df.rename(columns={'tic': 'ticker', 'datadate': 'date', 'cshtrm': 'volume', 'prccm': 'price'}) # rename columns
price_df['date'] = pd.to_datetime(price_df['date'], format='%Y-%m-%d')
# price_df.set_index('date', inplace=True)

In [218]:
# group = price_df.groupby('ticker')
# price_df['ret_1m']  = group['price'].transform(lambda x: x.pct_change())
# price_df['ret_3m']  = group['price'].transform(lambda x: x.pct_change(3))
# price_df['ret_6m']  = group['price'].transform(lambda x: x.pct_change(6))

# # 1. Simple Moving Averages
# price_df['SMA_3']  = group['ret_1m'].transform(lambda x: x.rolling(3).mean())
# price_df['SMA_6']  = group['ret_1m'].transform(lambda x: x.rolling(6).mean())

# # 3. Volatility (Rolling Std of returns)
# price_df['vol_3m']  = group['ret_1m'].transform(lambda x: x.rolling(3).std())
# price_df['vol_6m']  = group['ret_1m'].transform(lambda x: x.rolling(6).std())
# # price_df['vol_12m'] = group['ret_1m'].transform(lambda x: x.rolling(12).std())

# # 4. Exponential Moving Average
# price_df['EMA_6']  = group['ret_1m'].transform(lambda x: x.ewm(span=3, adjust=False).mean())
# price_df['EMA_12'] = group['ret_1m'].transform(lambda x: x.ewm(span=6, adjust=False).mean())


# # 7. Volume moving average
# price_df['vol_SMA_3'] = group['volume'].transform(lambda x: x.rolling(3).mean())
# price_df['vol_SMA_6'] = group['volume'].transform(lambda x: x.rolling(6).mean())

# # 8. Volume Rate of Change
# price_df['vol_ROC'] = group['volume'].transform(lambda x: x.pct_change())

# # 9. OBV (On-Balance Volume)
# def obv(df):
#     sign = df['price'].diff().apply(lambda x: 1 if x>0 else -1 if x<0 else 0)
#     return (sign * df['volume']).cumsum()

# price_df['OBV'] = group.apply(obv).reset_index(level=0, drop=True)


In [219]:
# Assuming price_df is set up
group = price_df.groupby('ticker')

# --- 1. Base Returns (Correct) ---
price_df['ret_1m']  = group['price'].transform(lambda x: x.pct_change())
price_df['ret_3m']  = group['price'].transform(lambda x: x.pct_change(3))
price_df['ret_6m']  = group['price'].transform(lambda x: x.pct_change(6))

# --- 2. SMA of Returns (Correct) ---
price_df['SMA_3']   = group['ret_1m'].transform(lambda x: x.rolling(3).mean())
price_df['SMA_6']   = group['ret_1m'].transform(lambda x: x.rolling(6).mean())

# --- 3. Volatility (Correct) ---
price_df['vol_3m']  = group['ret_1m'].transform(lambda x: x.rolling(3).std())
price_df['vol_6m']  = group['ret_1m'].transform(lambda x: x.rolling(6).std())

# --- 4. EMA of Returns (Fixed parameters) ---
# Fixed: span should match the variable name concept (or close to it)
price_df['EMA_3']   = group['ret_1m'].transform(lambda x: x.ewm(span=3, adjust=False).mean())
price_df['EMA_6']  = group['ret_1m'].transform(lambda x: x.ewm(span=6, adjust=False).mean())

# --- 5. Relative Volume (Fixed Volume SMA) ---
# logic: Current Volume / Average Volume
# If result > 1, volume is higher than average.
price_df['RVOL_3'] = group['volume'].transform(lambda x: x / x.rolling(3).mean())
price_df['RVOL_6'] = group['volume'].transform(lambda x: x / x.rolling(6).mean())

# --- 6. Volume Rate of Change (Correct) ---
price_df['vol_ROC'] = group['volume'].transform(lambda x: x.pct_change())

# --- 7. Volume-Weighted Momentum (Fixed OBV) ---
# Instead of cumulative OBV, we look at the immediate impact of volume on price.
# Logic: Return * Relative Volume. 
# A high return on high volume = Strong Signal.
price_df['vol_mom'] = price_df['ret_1m'] * price_df['RVOL_3']

# Final Cleanup
price_df.dropna(inplace=True)

In [220]:
price_df.isna().sum()

ticker     0
date       0
volume     0
price      0
ret_1m     0
ret_3m     0
ret_6m     0
SMA_3      0
SMA_6      0
vol_3m     0
vol_6m     0
EMA_3      0
EMA_6      0
RVOL_3     0
RVOL_6     0
vol_ROC    0
vol_mom    0
dtype: int64

In [221]:
price_df.columns

Index(['ticker', 'date', 'volume', 'price', 'ret_1m', 'ret_3m', 'ret_6m',
       'SMA_3', 'SMA_6', 'vol_3m', 'vol_6m', 'EMA_3', 'EMA_6', 'RVOL_3',
       'RVOL_6', 'vol_ROC', 'vol_mom'],
      dtype='object')

In [222]:
price_df.shape

(8641, 17)

In [223]:
nans_col = df.isna().sum()
missing_columns = nans_col[nans_col > 0]

for col in missing_columns.index:
    if col not in ['gind', 'gsector']:
        # 1. Fill using industry mean
        df[col] = df[col].fillna(df.groupby('gind')[col].transform('mean'))
        # 2. Fill remaining using sector mean
        df[col] = df[col].fillna(df.groupby('gsector')[col].transform('mean'))
        

In [224]:
# df.isna().sum()

In [225]:
# df.isna().sum()

In [226]:
# all other missing values imputed using KNN imputer
from sklearn.impute import KNNImputer

num_cols = df.select_dtypes(include='number').columns
imputer = KNNImputer(n_neighbors=3)
df[num_cols] = imputer.fit_transform(df[num_cols])

# still check for any missing values then simple fillna with column mean for numerical columns
df.drop(['gind', 'gsector'], axis=1, inplace=True)
num_cols = df.select_dtypes(include='number').columns
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())

In [227]:
# df.isna().sum()

In [228]:
price_tickers = price_df['ticker'].unique()
fundamentals_tickers = df['ticker'].unique()
common_tickers = set(price_tickers).intersection(set(fundamentals_tickers))
print(f"Number of unique tickers in fundamentals data: {len(fundamentals_tickers)}")
print(f"Number of unique tickers in price data: {len(price_tickers)}")
print(f"Number of common tickers: {len(common_tickers)}")

Number of unique tickers in fundamentals data: 466
Number of unique tickers in price data: 482
Number of common tickers: 419


In [229]:
# Need to manually check which tickers are missing in either dataset
missing_tickers = set(fundamentals_tickers) - common_tickers
print(f"Tickers in fundamentals data but missing in price data: {len(missing_tickers)}")

Tickers in fundamentals data but missing in price data: 47


In [230]:
price_df.shape, df.shape

((8641, 17), (5347, 62))

In [231]:
df = df.set_index(['date', 'ticker'])
price_df = price_df.set_index(['date', 'ticker'])
df_merged = df.join(price_df, how='inner')
df_merged.reset_index(inplace=True)

In [232]:
df_merged.shape, len(df_merged['ticker'].unique())

((4528, 77), 418)

In [233]:
# df_merged.isna().sum()

In [234]:
# df_merged.isna().sum()

In [235]:
# show tickers with a single missing values
missing_tickers = df_merged[df_merged.isna().any(axis=1)]['ticker'].unique()
missing_tickers # investigate these tickers

array([], dtype=object)

In [236]:
df.isna().sum()

gvkey              0
CAPEI              0
bm                 0
evm                0
pe_op_basic        0
pe_op_dil          0
pe_exi             0
pe_inc             0
ps                 0
pcf                0
dpr                0
npm                0
opmbd              0
opmad              0
gpm                0
ptpm               0
cfm                0
roa                0
roe                0
roce               0
efftax             0
aftret_eq          0
aftret_invcapx     0
aftret_equity      0
pretret_earnat     0
GProf              0
equity_invcap      0
debt_invcap        0
totdebt_invcap     0
capital_ratio      0
int_debt           0
int_totdebt        0
cash_lt            0
debt_at            0
debt_ebitda        0
short_debt         0
lt_debt            0
cash_debt          0
fcf_ocf            0
lt_ppent           0
dltt_be            0
debt_assets        0
debt_capital       0
de_ratio           0
cash_ratio         0
quick_ratio        0
curr_ratio         0
cash_conversi

In [237]:
# droping these tickers
df_merged = df_merged[~df_merged['ticker'].isin(missing_tickers)]
df_merged.isna().sum().sum()

0

In [238]:
# if above is not 0, see why

In [239]:
# save csv to be used in modeling
df_merged.to_csv(f'Data/sp500_cleaned_{year}.csv', index=False)

In [240]:
df_merged.shape

(4528, 77)

In [241]:
print(df_merged["ticker"].nunique())
print(year)

418
2018


In [242]:
# df_merged.columns

In [243]:
df_merged[price_df.columns].isna().sum()

volume     0
price      0
ret_1m     0
ret_3m     0
ret_6m     0
SMA_3      0
SMA_6      0
vol_3m     0
vol_6m     0
EMA_3      0
EMA_6      0
RVOL_3     0
RVOL_6     0
vol_ROC    0
vol_mom    0
dtype: int64